### Pandas에서 데이터 연산하기

numpy로 기본 산술 연산, 삼각함수, 지수, 로그함수 등 요소 단위의 연산을 빠르게 수행할 수 있다. Pandas는 Numpy로 부터 이 기능을 대부분 상속

- 단항 연산의 경우, 유니버설 함수가 결과물에 인덱스와 열 테이블을 보존 (유니버설 함수: 배열 안에 있는 데이터 원소 별로 연산을 수행하는 함수)
- 이항 연산의 경우, Pandas가 유니버설 함수에 객체를 전달할 때 인덱스를 자동으로 정렬

#### 인덱스 보존

In [4]:
import numpy as np
import pandas as pd

In [6]:
# 랜덤한 값을 만들기 위한 시드값을 지정하는 함수 RandomState(시드값)
rng = np.random.RandomState(42)
# 랜덤한 (정수)값을 추출하는 randint(시작, 끝, 갯수)
ser = pd.Series(rng.randint(0,10,4)) # 0에서 10까지 4개
ser

0    6
1    3
2    7
3    4
dtype: int32

In [7]:
df = pd.DataFrame(rng.randint(0,10,(3,4)),
                  columns=['A','B','C','D'])

In [8]:
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


In [12]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [10]:
np.exp(np.array([1.1,2.1,3.1,4.1]))

array([ 3.00416602,  8.16616991, 22.19795128, 60.3402876 ])

In [13]:
np.sin(df*np.pi/4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


계산 후에도 데이터 형태의 맥락이 유지 된다.

#### 인덱스 정렬

두개의 Series 또는 DataFrame객체에 이항 연산을 적용하는 경우 Pandas는 연산을 수행하는 과정에서 인덱스를 정렬

In [15]:
area = pd.Series({'Alaska': 1722321, 
                  'Texas':394842, 
                  'California':397393},
                 name='area')
population = pd.Series({
    'California':39283912, 
    'Texas':39481293, 
    'New York': 39581232},
    name='population')

In [16]:
population / area

Alaska              NaN
California    98.854061
New York            NaN
Texas         99.992638
dtype: float64

In [17]:
A = pd.Series([2,4,6], index=[0,1,2])
B = pd.Series([1,3,5], index=[1,2,3])

In [18]:
A+B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

##### 데이터프레임의 경우

In [20]:
A = pd.DataFrame(rng.randint(0,20,(2,2)), columns=list('AB'))
B = pd.DataFrame(rng.randint(0,10,(3,3)), columns=list('BAC'))

In [25]:
A

,A,B
0,1,11
1,5,1


In [26]:
B

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


연산할 때 부족한(매칭되지 않는) 요소에 값을 넣고 싶다면
fill_value 옵션으로 값을 지정한다.

In [32]:
fill = A.stack().mean()
df = A.add(B, fill_value=fill)

### 데이터 프레임을 CSV 파일로 내보내기, 읽어오기

#### 내보내기

데이터프레임에서 to_csv("파일이름.csv") 함수로 작성한 내용을 csv 파일로 저장할 수 있다. 파일이름을 입력할 때 확장명(.csv)를 생략하지 않는다.

In [36]:
df.to_csv('export2.csv')

#### 읽어오기

csv 파일에서 저장한 데이터를 데이터프레임으로 읽어오려면 read_csv('파일경로\파일이름.csv', index_col=0)함수를 활용, (index_col = 0은 0번째 행을 인덱스로 활용한다는 의미)

In [42]:
df_read = pd.read_csv('./export.csv', index_col=0)

In [43]:
df_read

,A,B,C
0,1.0,15.0,13.5
1,13.0,6.0,4.5
2,6.5,13.5,10.5


##### Quiz
다음과 같은 데이터 프레임을 만들고 두 좌표(left, top), (right, bottom)로 만들어지는 평면의 면적을 구하여 데이터 프레임의 빈 값을 체우세요

In [57]:
quiz = pd.read_csv("./quiz2.csv", index_col=0)

In [58]:
quiz

,지역,left,top,right,bottom,평면
1,A,0,1,4,5,NaN
2,B,2,6,6,7,NaN
3,C,6,1,7,4,NaN
4,D,3,5,5,9,NaN


In [59]:
w = np.array(quiz['right'].values - quiz['left'].values)

In [60]:
w

array([4, 4, 1, 2], dtype=int64)

In [66]:
h = quiz['bottom'].values - quiz['top'].values

h

array([4, 1, 3, 4], dtype=int64)

In [62]:
quiz['평면'] = w * h

In [63]:
quiz

,지역,left,top,right,bottom,평면
1,A,0,1,4,5,16
2,B,2,6,6,7,4
3,C,6,1,7,4,3
4,D,3,5,5,9,8


### 누락된 데이터 처리하기

누락된 데이터 존재를 나타내기 위한 여러 가지 방식

- 1비트를 누락 전용 표기로 활용 (P0012, N0012, P0014, P0015 ...)
- 데이터에서 나올 수 없는 값 (-1, -9999, 0101010)
- NaN(Not a Number)로 표시
- NA(Nat Available) : 값이 결손(손상)됨, Pandas에 미포함
- Null : 값이 존재하지 않는다. 손상되어 쓸 수 없는 경우도 포함

#### None : 파이썬의 누락된 데이터

None은 파이썬 코드에서 누락된 데이터 표시를 위해 사용, Pandas/Numpy 에서는 데이터 타입이 'object'일 경우에만 사용할 수 있다.

In [69]:
vals1 = np.array([1, 2, 3, 4])
vals1

array([1, 2, 3, 4])

In [70]:
vals2 = np.array([1, None, 3, 4])
vals2

array([1, None, 3, 4], dtype=object)

In [71]:
vals1.sum()

10

In [73]:
# vals2.sum()

#### NaN : 누락된 숫자 데이터

NaN(Not a Number)는 표준 IEEE 부동소수점 표기를 사용하는 모든 시스템이 인식하는 특수 부동 소수점 값이다

In [76]:
val2 = np.array([1, np.nan, 3, 4])
val2

array([ 1., nan,  3.,  4.])

NaN이 포함된 산술 연산의 결과는 항상 NaN

In [77]:
val2.sum()

nan

NaN 값을 무시하는 연산들

In [82]:
np.nansum(val2), np.nanmin(val2), np.nanmax(val2)

(8.0, 1.0, 4.0)

### Pandas에서 NaN과 None

NaN과 None은 각자의 역할이 있고 Pandas에서는 이 둘을 서로 호환 및 변환 할 수 있다

In [84]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

#### 널 값 연산하기

Pandas는 None과 NaN을 근본적으로 누락된 값이나 없는 값을 가리키기 위해 호환되는 값으로 처리한다. 데이터에서 이 값들을 감시하고 삭제하는데 유용한 메소드들이 있다

#### 널 값 감지 isnull(), notnull()

널 데이터를 탐지하기 위한 메소드로 데이터에 대한 불 배열을 반환한다

In [87]:
data = pd.Series([1, np.nan, 'hello', None])
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [88]:
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

#### 널 값 제거하기 dropna(), fillna()

NA값을 제거하는 dropna(), NA값을 채우는 fillna()

In [90]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [91]:
data.dropna()

0        1
2    hello
dtype: object

In [103]:
df = pd.DataFrame([[1, np.nan, 2],
                   [2, 3, 5],
                   [np.nan, 4, 6]])

In [104]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


데이터 프레임에서 dropna()는 데이터프레임에 대해 널 값이 있는 모든 행을 삭제한다.

In [105]:
df.dropna()

,0,1,2
1,2.0,3.0,5


In [106]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


열을 기준으로 삭제할 경우는 axis 옵션을 "columns"로 준다

In [107]:
df.dropna(axis="columns")

,2
0,2
1,5
2,6


In [108]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [109]:
df[3] = np.nan

In [110]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [111]:
df.dropna(axis="columns", how="all")

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [112]:
df.dropna(axis="columns", how="any")

,2
0,2
1,5
2,6


행이나 열의 값이 모두 NA값이거나 일부분만 NA일 경우 how 옵션을 쓴다, 
기본 how값은 any
- any : NA값이 있는 행이나 열을 모두 제거
- all : 행이나 열의 값이 모두 NA 값일 경우에 제거

In [99]:
df[3] = np.nan
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


thresh 옵션은 행이나 열에서 널이 아닌 값이 최소 몇 개 있는지 지정

In [114]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [113]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


#### 널 값 채우기 fillna()

NA 값을 삭제하지 않고 유효한 값으로 대체해야 할 때가 있다. Pandas는 이러한 연산을 위해 fillna()함수를 제공한다.

In [115]:
data = pd.Series(
    [1,np.nan,2,None,3],
    index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [116]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

이전 값으로 체우는 ffill(forward-fill), bfill(back-fill)과 기준 축 지정 axis(rows는 0 columns은 1)

In [117]:
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [118]:
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [119]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


axis에서 1, "columns"은 열을 의미하고, 0, "rows" 행을 의미한다. 

In [123]:
df.fillna(method='ffill', axis=0) 

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,2.0,4.0,6,NaN


##### Quiz
다음과 같이 데이터가 있을 때 데이터프레임을 만들고 아래와 같이 누락된 데이터를 처리하세요. (quiz1.csv 파일을 참조하세요)

In [128]:
quiz_load = pd.read_csv('./quiz1.csv', index_col=0)
quiz_load

,state,gender,name,win,lose
0,AK,NaN,Elsie,38,1.0
1,AL,NaN,Clyde,38,NaN
2,CA,0.0,Russell,32,NaN
3,FL,NaN,Eliza,32,NaN
4,HI,0.0,Charles,33,NaN
5,IA,NaN,Margaret,25,NaN


,state,name,win
0,AK,Elsie,38
1,AL,Clyde,38
2,CA,Russell,32
3,FL,Eliza,32
4,HI,Charles,33
5,IA,Margaret,25
